# 🤖 Agents e Tools: Dando "Superpoderes" para seus LLMs!

![](https://s3.us-east-1.amazonaws.com/turing.education/notebooks/imagens/langchain-modulo-11_img_01.png)

**Pedro Nunes Guth - Módulo 11 de 17: LangChain v0.3**

---

Tá, vamos falar sério aqui! Até agora nossos LLMs eram meio que aquele amigo inteligente, mas que só sabe conversar. E se eu te disser que podemos dar "ferramentas" para ele usar? 

Imagina o seguinte: você pergunta "Quanto é 2534 x 8971?" e em vez do LLM "chutar" o resultado, ele literalmente usa uma calculadora! Ou você pede "Me diga o clima em São Paulo" e ele consulta uma API de verdade!

**Isso é o que vamos aprender hoje: Agents e Tools!**

Bora! 🚀

## 🎯 O que são Agents?

Cara, pensa no Agent como aquele assistente super eficiente que não só conversa contigo, mas também **toma decisões** sobre quais ferramentas usar!

### Analogia do Garçom Inteligente 🍽️

Imagina um garçom em um restaurante que:
1. **Escuta** seu pedido
2. **Decide** se precisa consultar o cardápio, verificar estoque ou falar com o chef
3. **Usa as ferramentas** necessárias (tablet, telefone, vai até a cozinha)
4. **Te dá** a resposta completa

É isso que um Agent faz! Ele decide qual ferramenta usar baseado no que você perguntou.

### Diferença Crucial:

- **LLM simples**: "Acho que 2+2 = 4" (pode errar cálculos complexos)
- **Agent com calculadora**: Usa calculadora de verdade → "2+2 = 4" (sempre certo)

**Dica!** Agents são perfeitos quando você precisa de informações atuais ou cálculos precisos!

## ⚒️ O que são Tools?

Tools (ferramentas) são as "habilidades especiais" que você dá para o Agent usar!

### Tipos de Tools Populares:

1. **🧮 Calculadora** - Para matemática precisa
2. **🌐 Web Search** - Buscar info na internet
3. **📊 API Calls** - Consultar serviços externos
4. **📝 File Operations** - Ler/escrever arquivos
5. **🗃️ Database** - Consultar bancos de dados

### Analogia da Caixa de Ferramentas 🧰

É como dar uma caixa de ferramentas para um marceneiro:
- Tem um parafuso? Usa a chave de fenda
- Precisa cortar? Usa a serra
- Quer medir? Usa a régua

O Agent **escolhe automaticamente** qual ferramenta usar!

**Dica!** Cada tool tem uma "descrição" que ensina o Agent quando usá-la!

In [ ]:
# Bora começar! Instalando as dependências
!pip install langchain langchain-google-genai langchain-community python-dotenv -q

print("📦 Pacotes instalados com sucesso!")
print("🚀 Pronto para criar nossos primeiros Agents!")

In [ ]:
# Configuração inicial - Lembra dos módulos anteriores?
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.tools import Tool
from langchain_core.prompts import ChatPromptTemplate

# Carrega variáveis de ambiente
load_dotenv()

# Configurar a API key do Google (você já aprendeu isso no módulo 2!)
# Coloque sua chave aqui ou use .env
os.environ["GOOGLE_API_KEY"] = "SUA_CHAVE_AQUI"  # Substitua pela sua chave

# Criando nosso modelo (igual ao módulo 2)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0  # Zero para ser mais preciso
)

print("✅ LLM configurado!")
print("🎯 Modelo: gemini-2.0-flash (nosso queridinho do curso)")

## 🧮 Criando Nossa Primeira Tool: Calculadora

Vamos criar uma calculadora simples para mostrar como funciona!

### Como funciona uma Tool:

1. **Nome**: Como a ferramenta se chama
2. **Descrição**: Quando o Agent deve usá-la
3. **Função**: O que ela realmente faz

É tipo ensinar uma criança: "Esta ferramenta se chama 'calculadora', use ela quando precisar fazer contas, e ela funciona assim..."

In [ ]:
# Criando uma função simples de calculadora
def calculadora(expressao):
    """
    Função que calcula expressões matemáticas simples
    Exemplo: "2 + 2" retorna "4"
    """
    try:
        # eval() é meio perigoso em produção, mas para exemplo tá ok
        resultado = eval(expressao)
        return f"O resultado de {expressao} é {resultado}"
    except Exception as e:
        return f"Erro no cálculo: {str(e)}"

# Testando nossa função
print("🧪 Testando a calculadora:")
print(calculadora("2 + 2"))
print(calculadora("10 * 5"))
print(calculadora("100 / 4"))

In [ ]:
# Agora vamos transformar nossa função em uma Tool do LangChain
calculator_tool = Tool(
    name="Calculadora",
    description="Use esta ferramenta quando precisar fazer cálculos matemáticos. Entrada deve ser uma expressão matemática como '2+2' ou '10*5'.",
    func=calculadora
)

# Vamos criar mais uma tool útil: informações sobre o Brasil
def info_brasil(pergunta):
    """
    Banco de dados simples sobre o Brasil
    """
    info_db = {
        "capital": "A capital do Brasil é Brasília",
        "população": "O Brasil tem aproximadamente 215 milhões de habitantes", 
        "moeda": "A moeda do Brasil é o Real (BRL)",
        "idioma": "O idioma oficial do Brasil é o Português",
        "area": "O Brasil tem 8.515.767 km² de área"
    }
    
    # Busca a informação mais próxima
    pergunta_lower = pergunta.lower()
    for chave, resposta in info_db.items():
        if chave in pergunta_lower:
            return resposta
    
    return "Informação não encontrada. Posso ajudar com: capital, população, moeda, idioma, area"

# Criando a segunda tool
brasil_tool = Tool(
    name="Info_Brasil",
    description="Use esta ferramenta para obter informações básicas sobre o Brasil: capital, população, moeda, idioma, área.",
    func=info_brasil
)

print("🛠️ Tools criadas com sucesso!")
print("📋 Ferramentas disponíveis:")
print(f"1. {calculator_tool.name}: {calculator_tool.description}")
print(f"2. {brasil_tool.name}: {brasil_tool.description}")

## 🤖 Criando Nosso Primeiro Agent

Agora vem a parte mais legal! Vamos juntar tudo:
- **LLM** (já configurado)
- **Tools** (acabamos de criar)
- **Prompt** (você já sabe do módulo 4!)

### O Agent vai:
1. **Analisar** sua pergunta
2. **Decidir** qual tool usar (ou se não precisa de nenhuma)
3. **Executar** a tool se necessário
4. **Responder** com o resultado

![](https://s3.us-east-1.amazonaws.com/turing.education/notebooks/imagens/langchain-modulo-11_img_02.png)

In [ ]:
# Lista de todas as nossas tools
tools = [calculator_tool, brasil_tool]

# Criando o prompt do agent (lembra dos Prompt Templates do módulo 4?)
prompt = ChatPromptTemplate.from_messages([
    ("system", """
    Você é um assistente útil e inteligente.
    Você tem acesso a ferramentas específicas para ajudar o usuário.
    
    Use as ferramentas quando necessário:
    - Para cálculos matemáticos, use a Calculadora
    - Para informações sobre o Brasil, use Info_Brasil
    
    Se não precisar de nenhuma ferramenta, responda diretamente.
    Seja sempre educado e explique o que você está fazendo.
    """),
    ("user", "{input}"),
    ("assistant", "{agent_scratchpad}")
])

print("📝 Prompt do agent configurado!")
print("🎯 O agent sabe quando usar cada ferramenta")

In [ ]:
# Criando o agent (a mágica acontece aqui!)
agent = create_tool_calling_agent(llm, tools, prompt)

# Criando o executor (quem vai rodar o agent)
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,  # Para ver o que está acontecendo
    handle_parsing_errors=True
)

print("🎉 AGENT CRIADO COM SUCESSO!")
print("🔧 Ferramentas disponíveis:", [tool.name for tool in tools])
print("✅ Pronto para usar!")

## 🧪 Testando Nosso Agent

Hora do show! Vamos testar nosso agent com diferentes tipos de pergunta para ver ele escolhendo as ferramentas certas!

### Vamos testar:
1. **Cálculo matemático** → Deve usar a Calculadora
2. **Pergunta sobre Brasil** → Deve usar Info_Brasil  
3. **Pergunta geral** → Não deve usar nenhuma tool

**Dica!** Repare como o `verbose=True` vai mostrar o "raciocínio" do agent!

In [ ]:
# Teste 1: Cálculo matemático
print("🧮 TESTE 1: Cálculo Matemático")
print("="*50)

pergunta1 = "Quanto é 1234 * 5678?"
resposta1 = agent_executor.invoke({"input": pergunta1})

print(f"\n💬 Pergunta: {pergunta1}")
print(f"🤖 Resposta: {resposta1['output']}")
print("\n" + "="*50)

In [ ]:
# Teste 2: Informação sobre o Brasil
print("🇧🇷 TESTE 2: Informações sobre o Brasil")
print("="*50)

pergunta2 = "Qual é a capital do Brasil?"
resposta2 = agent_executor.invoke({"input": pergunta2})

print(f"\n💬 Pergunta: {pergunta2}")
print(f"🤖 Resposta: {resposta2['output']}")
print("\n" + "="*50)

In [ ]:
# Teste 3: Pergunta geral (não deve usar nenhuma tool)
print("💭 TESTE 3: Pergunta Geral")
print("="*50)

pergunta3 = "O que você acha do futebol brasileiro?"
resposta3 = agent_executor.invoke({"input": pergunta3})

print(f"\n💬 Pergunta: {pergunta3}")
print(f"🤖 Resposta: {resposta3['output']}")
print("\n" + "="*50)

## 📊 Visualizando Como o Agent Funciona

Bora criar um gráfico para entender melhor o fluxo de decisão do agent!

### Fluxo de Decisão:
```mermaid
graph TD
    A[Usuário faz pergunta] --> B[Agent analisa]
    B --> C{Precisa de tool?}
    C -->|Sim| D[Escolhe a tool certa]
    C -->|Não| E[Responde diretamente]
    D --> F[Executa a tool]
    F --> G[Processa resultado]
    G --> H[Resposta final]
    E --> H
```

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Simulando o tempo de resposta para diferentes tipos de pergunta
tipos_pergunta = ['Cálculo\nMatemático', 'Info\nBrasil', 'Pergunta\nGeral']
tempos_resposta = [2.3, 1.8, 1.2]  # segundos simulados
usa_tool = ['Sim', 'Sim', 'Não']

# Cores diferentes para cada tipo
cores = ['#FF6B6B', '#4ECDC4', '#45B7D1']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Tempo de resposta
bars1 = ax1.bar(tipos_pergunta, tempos_resposta, color=cores, alpha=0.7, edgecolor='black')
ax1.set_title('⏱️ Tempo de Resposta por Tipo de Pergunta', fontsize=14, pad=20)
ax1.set_ylabel('Tempo (segundos)', fontsize=12)
ax1.set_ylim(0, 3)

# Adicionando valores no topo das barras
for i, bar in enumerate(bars1):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.05,
             f'{height}s', ha='center', va='bottom', fontweight='bold')

# Gráfico 2: Uso de ferramentas
usa_tool_count = [usa_tool.count('Sim'), usa_tool.count('Não')]
labels = ['Usa Tool', 'Não Usa Tool']
colors_pie = ['#FF9999', '#66B2FF']

wedges, texts, autotexts = ax2.pie(usa_tool_count, labels=labels, colors=colors_pie, 
                                   autopct='%1.1f%%', startangle=90)
ax2.set_title('🛠️ Distribuição de Uso de Ferramentas', fontsize=14, pad=20)

plt.tight_layout()
plt.show()

print("📈 Análise dos resultados:")
print("✅ Perguntas que precisam de tools demoram um pouco mais")
print("✅ Agent conseguiu identificar quando usar cada ferramenta")
print("✅ Respostas diretas são mais rápidas")

## 🌐 Criando uma Tool Mais Avançada: Web Search

Agora vamos criar algo mais poderoso! Uma ferramenta que simula busca na web.

**Atenção**: Em produção você usaria APIs reais como Google Search API, mas vamos simular para aprender o conceito!

### Por que Web Search é importante?
- **Informações atuais**: LLMs têm data de corte de treinamento
- **Fatos específicos**: Preços, notícias, dados em tempo real
- **Verificação**: Confirmar informações importantes

![](https://s3.us-east-1.amazonaws.com/turing.education/notebooks/imagens/langchain-modulo-11_img_03.png)

In [ ]:
# Simulando uma base de "web search" com informações atualizadas
def web_search_simulada(query):
    """
    Simula uma busca na web com informações "atuais"
    Em produção, isso seria uma API real do Google, Bing, etc.
    """
    
    # Base de dados simulada com informações "recentes"
    resultados_web = {
        "clima": "Hoje em São Paulo: 28°C, parcialmente nublado. Fonte: Weather API",
        "dólar": "Cotação atual do dólar: R$ 5,85. Última atualização: hoje 15h30",
        "copa": "Próxima Copa do Mundo será em 2026 nos EUA, Canadá e México",
        "bitcoin": "Bitcoin hoje: $43,250 USD (+2.3% nas últimas 24h)",
        "eleições": "Próximas eleições municipais no Brasil: 2024",
        "feriado": "Próximo feriado nacional: Carnaval (fevereiro 2024)"
    }
    
    query_lower = query.lower()
    
    # Busca por palavras-chave
    for palavra_chave, resultado in resultados_web.items():
        if palavra_chave in query_lower:
            return f"🔍 Resultado da busca: {resultado}"
    
    # Se não encontrar nada específico
    return "🔍 Busca realizada, mas não encontrei informações específicas sobre isso. Posso ajudar com: clima, dólar, copa, bitcoin, eleições, feriado"

# Criando a tool de web search
web_search_tool = Tool(
    name="Web_Search",
    description="Use esta ferramenta para buscar informações atuais na internet como clima, cotações, notícias recentes, etc. Útil quando o usuário pergunta sobre dados em tempo real.",
    func=web_search_simulada
)

# Testando a nova tool
print("🌐 Testando a Web Search:")
print(web_search_simulada("qual a cotação do dólar hoje?"))
print(web_search_simulada("como está o clima em São Paulo?"))

In [ ]:
# Criando um Agent mais poderoso com 3 tools!
tools_avancadas = [calculator_tool, brasil_tool, web_search_tool]

# Prompt mais elaborado
prompt_avancado = ChatPromptTemplate.from_messages([
    ("system", """
    Você é um assistente super inteligente com acesso a ferramentas especiais!
    
    🧮 CALCULADORA: Para qualquer cálculo matemático
    🇧🇷 INFO_BRASIL: Para informações básicas sobre o Brasil
    🌐 WEB_SEARCH: Para informações atuais (clima, cotações, notícias)
    
    INSTRUÇÕES:
    1. Analise a pergunta do usuário
    2. Decida qual ferramenta usar (ou nenhuma)
    3. Execute a ferramenta se necessário
    4. Dê uma resposta completa e amigável
    
    Seja sempre educado e explique o que você fez!
    """),
    ("user", "{input}"),
    ("assistant", "{agent_scratchpad}")
])

# Criando o agent avançado
agent_avancado = create_tool_calling_agent(llm, tools_avancadas, prompt_avancado)

agent_executor_avancado = AgentExecutor(
    agent=agent_avancado,
    tools=tools_avancadas,
    verbose=True,
    handle_parsing_errors=True
)

print("🚀 AGENT AVANÇADO CRIADO!")
print(f"🔧 {len(tools_avancadas)} ferramentas disponíveis:")
for i, tool in enumerate(tools_avancadas, 1):
    print(f"  {i}. {tool.name}")

## 🎯 Testando o Agent Avançado

Agora vamos ver nosso agent em ação com mais ferramentas! Ele precisa ser ainda mais inteligente para escolher a ferramenta certa.

**Dica!** Observe como ele vai "raciocinar" sobre qual tool usar em cada situação!

In [ ]:
# Teste com múltiplas perguntas para ver a escolha de tools
perguntas_teste = [
    "Quanto é 987 + 654 - 321?",
    "Qual é a cotação do dólar hoje?", 
    "Quantos habitantes tem o Brasil?",
    "Como está o clima em São Paulo?",
    "Qual sua cor favorita?"  # Esta não deve usar nenhuma tool
]

print("🧪 BATERIA DE TESTES DO AGENT AVANÇADO")
print("="*60)

for i, pergunta in enumerate(perguntas_teste, 1):
    print(f"\n🔸 TESTE {i}: {pergunta}")
    print("-" * 40)
    
    try:
        resposta = agent_executor_avancado.invoke({"input": pergunta})
        print(f"✅ Resposta: {resposta['output']}")
    except Exception as e:
        print(f"❌ Erro: {e}")
    
    print("-" * 40)

## 🔄 Como Funciona o Reasoning do Agent

Tá, mas como o agent "decide" qual ferramenta usar? Vamos desvendar essa mágica!

### O Processo de Decisão:

1. **📝 Análise da Pergunta**: O LLM lê e entende o que você quer
2. **🤔 Reasoning**: "Hm, isso precisa de cálculo? Busca na web? Info específica?"
3. **⚡ Seleção da Tool**: Escolhe baseado na descrição de cada ferramenta
4. **🔧 Execução**: Roda a ferramenta com os parâmetros corretos
5. **📊 Processamento**: Analisa o resultado da ferramenta
6. **💬 Resposta Final**: Formata uma resposta amigável

### Analogia do GPS 🗺️
É como um GPS que:
- Analisa seu destino
- Verifica o trânsito atual
- Escolhe a melhor rota
- Te guia passo a passo

**Dica!** A qualidade da "description" de cada tool é crucial para o agent escolher certo!

In [ ]:
# Vamos criar um exemplo que mostra o "raciocínio" do agent mais claramente
import time

def mostrar_reasoning(pergunta):
    """
    Função para demonstrar o processo de reasoning do agent
    """
    print(f"🤖 ANALISANDO: '{pergunta}'")
    print("\n🧠 Processo de Reasoning:")
    
    # Simulando o processo mental
    time.sleep(0.5)
    print("1️⃣ Lendo e entendendo a pergunta...")
    
    time.sleep(0.5)
    print("2️⃣ Verificando se preciso de alguma ferramenta...")
    
    # Análise simples baseada em palavras-chave
    pergunta_lower = pergunta.lower()
    
    if any(op in pergunta_lower for op in ['+', '-', '*', '/', 'calcul', 'quanto é']):
        print("3️⃣ 🧮 Detectei matemática! Vou usar a Calculadora")
        tool_escolhida = "Calculadora"
    elif any(palavra in pergunta_lower for palavra in ['cotação', 'dólar', 'clima', 'bitcoin']):
        print("3️⃣ 🌐 Detectei busca por info atual! Vou usar Web Search")
        tool_escolhida = "Web_Search"
    elif any(palavra in pergunta_lower for palavra in ['brasil', 'capital', 'população', 'moeda']):
        print("3️⃣ 🇧🇷 Detectei pergunta sobre Brasil! Vou usar Info_Brasil")
        tool_escolhida = "Info_Brasil"
    else:
        print("3️⃣ 💭 Não preciso de ferramentas, posso responder diretamente")
        tool_escolhida = "Nenhuma"
    
    time.sleep(0.5)
    print(f"4️⃣ ✅ Decisão tomada: {tool_escolhida}")
    print("\n" + "="*50)

# Testando o processo de reasoning
exemplos = [
    "Quanto é 15 * 8?",
    "Qual a capital do Brasil?",
    "Como está o dólar hoje?",
    "Qual sua comida favorita?"
]

for exemplo in exemplos:
    mostrar_reasoning(exemplo)
    print()

## 💡 Exercícios Práticos

Agora é sua vez! Vamos praticar criando tools e testando nosso agent.

### 🎯 Exercício 1: Criando uma Tool de Conversão
Crie uma tool que converte temperaturas (Celsius ↔ Fahrenheit)

### 🎯 Exercício 2: Testando Edge Cases
Teste o agent com perguntas "maliciosas" ou ambíguas

### 🎯 Exercício 3: Tool Personalizada
Crie uma tool com informações sobre sua cidade/região

**Dica!** Comece simples e vá aumentando a complexidade!

In [ ]:
# EXERCÍCIO 1: Sua vez! Crie uma tool de conversão de temperatura

def conversor_temperatura(entrada):
    """
    Converte temperaturas entre Celsius e Fahrenheit
    Exemplos de entrada: "25C", "77F", "0 celsius", "32 fahrenheit"
    
    Complete esta função!
    """
    # SEU CÓDIGO AQUI!
    # Dicas:
    # - C para F: (C * 9/5) + 32
    # - F para C: (F - 32) * 5/9
    # - Use .upper(), .replace(), split() para processar a entrada
    
    try:
        entrada_clean = entrada.upper().replace(" ", "")
        
        if "C" in entrada_clean:
            # Extrair número e converter para Fahrenheit
            numero = float(entrada_clean.replace("C", "").replace("CELSIUS", ""))
            fahrenheit = (numero * 9/5) + 32
            return f"{numero}°C = {fahrenheit:.1f}°F"
            
        elif "F" in entrada_clean:
            # Extrair número e converter para Celsius  
            numero = float(entrada_clean.replace("F", "").replace("FAHRENHEIT", ""))
            celsius = (numero - 32) * 5/9
            return f"{numero}°F = {celsius:.1f}°C"
            
        else:
            return "Formato inválido. Use: '25C', '77F', '0 celsius' ou '32 fahrenheit'"
            
    except Exception as e:
        return f"Erro na conversão: {str(e)}"

# Teste sua função
print("🧪 Testando o conversor:")
print(conversor_temperatura("25C"))
print(conversor_temperatura("77F"))
print(conversor_temperatura("0 celsius"))

In [ ]:
# EXERCÍCIO 2: Criando a tool e adicionando ao agent

# Crie a tool de temperatura
temperatura_tool = Tool(
    name="Conversor_Temperatura",
    description="Use para converter temperaturas entre Celsius e Fahrenheit. Aceita formatos como '25C', '77F', 'celsius', 'fahrenheit'.",
    func=conversor_temperatura
)

# Agent com 4 tools agora!
todas_tools = [calculator_tool, brasil_tool, web_search_tool, temperatura_tool]

# Criando agent final
agent_final = create_tool_calling_agent(llm, todas_tools, prompt_avancado)

agent_executor_final = AgentExecutor(
    agent=agent_final,
    tools=todas_tools,
    verbose=True,
    handle_parsing_errors=True
)

print("🎉 AGENT FINAL CRIADO!")
print(f"🔧 Agora temos {len(todas_tools)} ferramentas:")
for i, tool in enumerate(todas_tools, 1):
    print(f"  {i}. {tool.name}")

# Teste a nova funcionalidade
print("\n🧪 Testando conversão de temperatura:")
resultado = agent_executor_final.invoke({"input": "Converte 30 graus celsius para fahrenheit"})
print(f"Resultado: {resultado['output']}")

## 📊 Comparação: Com vs Sem Agents

Vamos ver a diferença prática entre usar um LLM simples vs um Agent com tools!

![](https://s3.us-east-1.amazonaws.com/turing.education/notebooks/imagens/langchain-modulo-11_img_04.png)

### Teste Prático: Cálculo Complexo

In [ ]:
# Comparação: LLM simples vs Agent com tools

pergunta_complexa = "Quanto é (1234 * 5678) + (9876 / 4) - 321?"

print("🥊 LLM SIMPLES vs AGENT COM TOOLS")
print("="*60)

# 1. Teste com LLM simples (sem tools)
print("\n🤖 LLM SIMPLES (sem ferramentas):")
print("-" * 40)
resposta_simples = llm.invoke(pergunta_complexa)
print(f"Resposta: {resposta_simples.content}")

# 2. Teste com Agent (com calculadora)
print("\n🦾 AGENT COM TOOLS:")
print("-" * 40)
resposta_agent = agent_executor_final.invoke({"input": pergunta_complexa})
print(f"Resposta: {resposta_agent['output']}")

# Verificação manual do cálculo
resultado_correto = (1234 * 5678) + (9876 / 4) - 321
print(f"\n✅ Resultado correto: {resultado_correto}")
print("\n📊 ANÁLISE:")
print("• LLM simples: Pode "chutar" ou errar cálculos complexos")
print("• Agent com tools: Usa calculadora real = sempre correto")

## 🚀 Preparando para os Próximos Módulos

Liiiindo! Você acabou de dominar Agents e Tools! Mas isso é só o começo...

### 🔮 O que vem por aí:

**Módulo 12-13: Projetos Finais**
- Vamos usar Agents em projetos reais!
- Combinar com RAG (módulo 10) + Memory (módulo 7)
- Chatbots inteligentes que fazem coisas de verdade

**Módulo 14: Deploy e Produção**
- Colocar seus Agents no ar com Streamlit
- Interface bonita para usuários finais

**Módulo 16: LangGraph**
- Agents ainda mais sofisticados
- Fluxos complexos de decisão
- Multi-agents trabalhando juntos!

### 💡 Conexões com módulos anteriores:
- **Prompt Templates** (Módulo 4): Essenciais para configurar agents
- **Memory** (Módulo 7): Agents que lembram de conversas passadas
- **RAG** (Módulo 10): Agents que consultam documentos específicos

**Dica!** Agents são o "cérebro" que conecta tudo que aprendemos até agora!

In [ ]:
# Visualização final: Evolução dos nossos LLMs no curso
import matplotlib.pyplot as plt
import numpy as np

modulos = ['Módulo 2\nChatModel', 'Módulo 4\nPrompt Template', 'Módulo 7\nMemory', 'Módulo 10\nRAG', 'Módulo 11\nAgents']
capacidades = [1, 2, 4, 7, 9]  # Nível de capacidade (1-10)
complexidade = [1, 2, 3, 6, 8]  # Nível de complexidade

fig, ax = plt.subplots(figsize=(12, 8))

x = np.arange(len(modulos))
width = 0.35

bars1 = ax.bar(x - width/2, capacidades, width, label='Capacidades do LLM', 
               color='#4CAF50', alpha=0.8)
bars2 = ax.bar(x + width/2, complexidade, width, label='Complexidade de Uso', 
               color='#FF9800', alpha=0.8)

ax.set_xlabel('Módulos do Curso', fontsize=12)
ax.set_ylabel('Nível (1-10)', fontsize=12)
ax.set_title('🚀 Evolução das Capacidades dos LLMs no Curso', fontsize=16, pad=20)
ax.set_xticks(x)
ax.set_xticklabels(modulos, rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3)

# Adicionando valores nas barras
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("📈 EVOLUÇÃO DO CURSO:")
print("✅ Começamos com LLMs básicos")
print("✅ Adicionamos templates e memory")
print("✅ Incorporamos knowledge com RAG")
print("✅ AGORA: Agents que usam ferramentas!")
print("🚀 PRÓXIMO: Projetos reais combinando tudo!")

## 📚 Resumo do Módulo: O que Aprendemos?

Cara, que jornada! Vamos recapitular o que dominamos sobre **Agents e Tools**:

### 🎯 Conceitos Fundamentais:
- **Agent**: O "cérebro" que decide quais ferramentas usar
- **Tools**: As "habilidades especiais" que damos para o LLM
- **Reasoning**: Como o agent "pensa" e toma decisões

### 🛠️ Na Prática Criamos:
1. **Calculadora Tool** - Matemática precisa
2. **Info Brasil Tool** - Base de dados local
3. **Web Search Tool** - Informações "atuais"
4. **Conversor Temperatura** - Funcionalidade personalizada

### 🧠 Processo de Decisão do Agent:
```
Pergunta → Análise → Escolha da Tool → Execução → Resposta
```

### 💡 Principais Aprendizados:
- **Descrições claras** das tools são cruciais
- Agents **combinam** inteligência com **precisão**
- **verbose=True** ajuda a entender o processo
- Tools podem ser **simples ou complexas**

### 🔗 Conexões com o Curso:
- Usamos **ChatModel** (Módulo 2)
- Aplicamos **Prompt Templates** (Módulo 4)
- Preparamos terreno para **projetos finais**

**Dica Final!** Agents são o futuro dos LLMs - eles não só "sabem", mas também "fazem"!

---

**🎉 Parabéns! Você agora sabe criar LLMs que usam ferramentas de verdade!**

**Próxima parada: Módulo 12 - Projeto Final 1** 🚀

Bora!